In [1]:
from dotenv import load_dotenv
import os
import numpy as np
import html


In [2]:
def prepare_xml(data):
    start_tag = "<diagram"
    end_tag = "</diagram>"
    start_index = data.find(start_tag)
    end_index = data.find(end_tag) + len(end_tag)
    xml_data = data[start_index:end_index]
    return xml_data

# Extract data from XML


def get_xml_cell_data(mx_cell):
    vertex = mx_cell.get("vertex", '')
    edge = mx_cell.get("edge", '')
    id = mx_cell.get("id", '')
    parent = html.unescape(mx_cell.get("parent", ''))
    source = html.unescape(mx_cell.get("source", ''))
    target = html.unescape(mx_cell.get("target", ''))
    value = html.unescape(mx_cell.get("value", '')).replace(" ", " ")
    styles = html.unescape(mx_cell.get("style", '')).split(";")
    return vertex, edge, id, parent, source, target, value, styles

# compile block


def abstract_block_compiler(shape, id, value):
    return {
        'mode': shape,
        'ins': {},
        'outs': {},
        'id': id,
        'value': value,
        'msg': '',
        'cross': False,
        'loops': [],
        'implemented': False,
        'name': '',
    }


def search_in_list(list, keyword):
    for item in list:
        if keyword in item:
            return item
    return False


In [3]:
def dfs(graph, block, visited, path, loops):
    visited[block['id']] = True
    path.append(block['id'])

    for neighbor_id in block['outs']:
        neighbor = graph[neighbor_id]
        if not visited[neighbor['id']]:
            dfs(graph, neighbor, visited, path, loops)
        elif neighbor['id'] in path:
            loop = tuple(path[path.index(neighbor['id']):])
            loops.append(loop)
    path.pop()
    visited[block['id']] = False


def figure_loops_area(blocks):
    global loops
    visited = {block: False for block in blocks}

    # Perform DFS from each unvisited node
    for block_id, block in blocks.items():
        if not visited[block_id]:
            path = []
            dfs(blocks, block, visited, path, loops)
    
    duplicate_free_loops = [loops[0]]
    loopsdetailed = []
    for loop in loops:
        allowed = True
        for checked_loop in duplicate_free_loops:
            look_for_similarities = False
            compare_lengthes = len(loop) == len(checked_loop)
            if compare_lengthes: look_for_similarities = np.array([np.where(np.array(checked_loop) == block_id) for block_id in loop]).flatten().size == len(checked_loop)
            if compare_lengthes and look_for_similarities:
                allowed = False
                break
        if allowed == True: duplicate_free_loops.append(loop)
    loops = duplicate_free_loops
    # Go through loops to see if they have decision blocks and setup the start of the loop
    blocks_iterated = []
    for loop in loops:
        loopindex = loops.index(loop)
        newloop = {
            'decision': False,
            'items': list(loop),
            'label': str(loopindex),
            'zero_point': list(loop)[0],
        }
        # look for decision block in loop
        # the starting block of loop can be dynamic, it just can't be Decision or Input block since their edges are important data
        decision_found = None
        for block_id in loop:
            look_for_block_in_previous_loops = len(np.where(np.array(blocks_iterated) == block_id)[0])
            if blocks[block_id]['mode'] == 'decision' and look_for_block_in_previous_loops == 0:
                decision_found = block_id
                break
        blocks_iterated += loop

        if decision_found != None:
            newloop['decision'] = True
            starting_block_index = loop.index(decision_found)+1
            newloop['zero_point'] = loop[starting_block_index]
            shiftedarray = list(np.roll(loop, -(starting_block_index)))
            newloop['items'] = shiftedarray
        
        loopsdetailed.append(newloop)

    return loopsdetailed

# def figure_decisions(blocks):
#     decisions = []
#     for block in blocks:
#         decision_block = blocks[block]
#         if decision_block['mode'] == 'decision':
#             for incoming in decision_block['ins']:
#                 prev_block = blocks[decision_block['ins'][incoming]['id']]

#                 decisions.append({
#                     'name': 'decision'+decision_block['id']+'_'+prev_block['id'],
#                     'condition': decision_block['value'],
#                     'zero_block': prev_block,
#                     'block': decision_block
#                 })
#     return decisions

In [4]:
# compile all shapes
def compile_shapes(root):
    global blocks, loops
    edges = {}
    edges_msg = {}
    the_start_block_id = None
    for mx_cell in root.findall(".//mxCell"):
        vertex, edge, id, parent, source, target, value, styles = get_xml_cell_data(
            mx_cell)
        # data = {}

        # edgelabel_check = search_in_list(styles, "edgeLabel")
        # edgestyle_check = search_in_list(styles, "edgeStyle")
        shape_check = search_in_list(styles, "shape")
        style_check = search_in_list(styles, "absoluteArcSize")

        # It's shape
        if shape_check != False or style_check != False:

            shape = shape_check if shape_check != False else style_check
            # data
            if shape == 'shape=parallelogram':
                shape = 'input'
            # display
            elif shape == "shape=mxgraph.flowchart.display":
                shape = 'display'
            # start
            elif shape == "shape=mxgraph.flowchart.start_2":
                shape = 'start'
                the_start_block_id = id
            # stored_data
            elif shape == "shape=mxgraph.flowchart.stored_data":
                shape = 'stored_data'
            # decision
            elif shape == "shape=mxgraph.flowchart.decision":
                shape = 'decision'
            # process
            elif shape == "absoluteArcSize=1":
                shape = 'process'

            blocks[id] = abstract_block_compiler(shape, id, value)

        # It's edge or sth
        else:
            msg = search_in_list(styles, "edgeLabel")
            if msg != False:
                edges_msg[id] = {'parent': parent, 'id': id, 'value': value}
            elif edge == '1':
                edges[id] = {'id': id, 'source': source,
                             'target': target, 'value': ''}

        continue

    # fuse edges with their messages
    for msg in edges_msg:
        target_edge_id = edges_msg[msg]['parent']
        edge_msg = edges_msg[msg]['value']
        edges[target_edge_id]['value'] = edge_msg

    # fuse blocks with their neighboor blocks (source/destination)
    for edge in edges:
        this_edge = edges[edge]
        the_source_block = blocks[this_edge['source']]
        the_target_block = blocks[this_edge['target']]

        the_source_block['outs'][the_target_block['id']] = {
            'id': the_target_block['id'],
            'value': the_target_block['value'],
            'msg': this_edge['value'],
            'cross': the_target_block['cross'],
            'loops': the_target_block['loops'],
            'mode': the_target_block['mode'],
        }
        the_target_block['ins'][the_source_block['id']] = {
            'id': the_source_block['id'],
            'value': the_source_block['value'],
            'msg': this_edge['value'],
            'cross': the_source_block['cross'],
            'loops': the_source_block['loops'],
            'mode': the_source_block['mode'],
        }

    # figure crossovers
    for block in blocks:
        this_block = blocks[block]
        if len(this_block['ins']) > 1:
            this_block['cross'] = True

    # figure loops
    loops = figure_loops_area(blocks)
    # decisions = figure_decisions(blocks)
    

    return blocks, loops, the_start_block_id


In [5]:
# Check if the dst_block is in the src_blocks flow
def flow_ctrl(blocks, dst_block_id, src_block_id):
    return search_in_list(blocks[src_block_id]['outs'], dst_block_id) != False

# follow the tail from the starting block


def generate_flow(init_block_id):
    global loops, blocks
    block_flow = []
    block_flow.append(blocks[init_block_id])
    for i in blocks:
        for block_id in blocks:
            is_block_of_loop_start = len(np.where(np.array([loop['items'][0] for loop in loops]) == block_id)[0])
            if is_block_of_loop_start == 0 and flow_ctrl(blocks, block_id, block_flow[-1]['id']) == True:
                block_flow.append(blocks[block_id])
    return block_flow


def detect_loop(init_block_id, blocks):
    the_flow = generate_flow(init_block_id)
    the_flow.pop(0)
    loop_found = False
    for block in the_flow:
        false_option = block.get('false', False)
        if init_block_id == block['id']:
            loop_found = True
        elif false_option != False and false_option['id'] == init_block_id:
            loop_found = True
    return loop_found


# Syntaxes


In [6]:
def input_block(suffix, block, prev_block_id):
    lines = ''
    lines += suffix + block['value']
    lines += " = "
    lines += "input("+block['ins'][prev_block_id]['value']+")"
    lines += "\n"
    return lines


def process_block(suffix, block):
    lines = ''
    lines += suffix+block['value']
    lines += "\n"
    return lines


def display_block(suffix, block):
    lines = ''
    lines += suffix+"print("
    lines += block['value']
    lines += ")\n"
    return lines


def stored_data_block(suffix, block):
    lines = ''
    lines += suffix+block['msg']
    lines += " = np.array(["
    for item in block['value'].split(','):
        lines += '"'+item.strip()+'", '
    lines += "])\n"
    return lines


def decision_block(suffix, block, flow):
    global blocks, compile_block, dealing_decisions, done, in_loop, actual_lines, in_decision, decision_refered
    if in_loop != False:
        actual_lines.append(suffix+block['name']+'('+block['value']+')\n')
    else:
        dealing_decisions.append(block['id'])
        condition_count = -1
        for dest_block_id in block['outs']:
            condition_count += 1
            dest_block = block['outs'][dest_block_id]

            # define the condition
            condition = block['value']

            if dest_block['msg'] == '':
                compile_block(blocks[dest_block_id], flow, suffix+"\t\t")
            else:
                # Write the condition
                if condition_count == 0 and dest_block['msg'] != '': actual_lines.append(suffix+"\t"+"if "+ condition + " " + dest_block['msg']+":\n")
                elif condition_count > 0 and dest_block['msg'] != '': actual_lines.append(suffix+"\t"+"elif "+ condition + " " + dest_block['msg']+":\n")
                elif dest_block['msg'] == 'else': actual_lines.append(suffix+"\t"+"else:\n")
                else: actual_lines.append(suffix+"\t"+"else:\n")

                # Write Decision Body
                # generate_flow(dest_block_id, blocks)
                # TODO: it's not compiling the full body
                prev_block_id = block['id']
                tmp_dest_block_id = dest_block_id
                while blocks[tmp_dest_block_id]['mode'] != 'decision' and done == False:
                    the_block = blocks[tmp_dest_block_id]
                    compile_block(the_block, flow, prev_block_id, suffix+"\t\t")
                    blocks[tmp_dest_block_id]['implemented'] = True
                    prev_block_id = tmp_dest_block_id
                    get_tmp_dest_block_id = list(blocks[tmp_dest_block_id]['outs'].items())[0][1]['id']
                    tmp_dest_block_id = get_tmp_dest_block_id
                    print("sasho")
                # if blocks[dest_block_id]['mode'] == 'decision':
    in_decision = False
    decision_refered = False


def decision_area(current_block, decisionblock, prev_block_id, flow):
    global blocks, compile_block, loops, decision_refered, actual_lines, in_decision, decisions
    decision_refered = decisionblock['id']
    blocks[decisionblock['id']]['name'] = 'decision'+str(len(dealing_decisions))
    # actual_lines.append('def '+blocks[decisionblock['id']]['name']+'('+decisionblock['value']+'=None):\n')
    in_decision = True
    decisions[str(len(dealing_decisions))] = [
        'def '+blocks[decisionblock['id']]['name']+'('+decisionblock['value']+'=None):\n'
    ]
    compile_block(current_block, flow, prev_block_id, '\t', store=decisions[str(len(dealing_decisions))])


In [7]:
# def define_decision_funcs():
#     global decisions, in_decision, actual_lines
#     for decision in decisions:
#         in_decision = True
#         decision_area(decision['zero_block'], decision['block'], )
#         print(actual_lines)
# # define_decision_funcs()

In [8]:
def compile_block(block, flow, prev_block_id, suffix='', store=False):
    global loop_refered, decision_refered, loops, done, in_loop, in_decision, actual_lines
    theline = ''
    # if block['implemented'] == True and loop_refered != False and len(np.where(np.array([loop['zero_point'] for loop in loops]) == block['id'])[0]) > 0 and loop_refered == str(np.where(np.array([loop['zero_point'] for loop in loops]) == block['id'])[0][0]):
    if len(np.where(np.array([block['implemented'] for block in flow]) == True)[0]) == len(flow):
        done = True
    if block['implemented'] == False:
        # self looped
        if loop_refered != False and len(np.where(np.array([loop['zero_point'] for loop in loops]) == block['id'])[0]) > 0 and in_loop != str([loop['items'][0] for loop in loops].index(block['id'])):    
            loop_refered = False
        # block is the one before decision block
        if in_loop == False and in_decision == False and block['mode'] != 'decision' and list(block['outs'].items())[0][1]['mode'] == 'decision':
            decision_area(block, list(block['outs'].items())[0][1], prev_block_id, flow)
        elif list(block['outs'].items())[0][1]['id'] != decision_refered and in_loop == False and in_decision == True and block['mode'] != 'decision' and list(block['outs'].items())[0][1]['mode'] == 'decision':
            theline = suffix+'decision'+str(len(dealing_decisions))+'('+blocks[list(block['outs'].items())[0][1]['id']]['value']+')\n'
            if store == False: actual_lines.append(theline)
            else: store.append(theline)

        # block is a loop item
        # if so, are we writing the loop itself or referreing to it?
        # referring to it; for the first time
        elif loop_refered == False and len(np.where(np.array([loop['items'][0] for loop in loops]) == block['id'])[0]) > 0 and in_loop != str([loop['items'][0] for loop in loops].index(block['id'])):
            loop_refered = str([loop['items'][0] for loop in loops].index(block['id']))
            theline = suffix+"loop"+loop_refered+"()\n"
            if store == False: actual_lines.append(theline)
            else: store.append(theline)
        # referring to it; the rest of the loop
        elif loop_refered != False and np.where(np.array(loops[int(loop_refered)]['items']) == block['id'])[0].size > 0:
            theline = ''
            if store == False: actual_lines.append(theline)
            else: store.append(theline)
        # defining it
        else:
            loop_refered = False
            if block['mode'] == 'input': theline = input_block(suffix, block, prev_block_id)
            elif block['mode'] == 'process': theline = process_block(suffix, block)
            elif block['mode'] == 'display': theline = display_block(suffix, block)
            elif block['mode'] == 'stored_data': theline = stored_data_block(suffix, block)
            elif block['mode'] == 'decision': theline = decision_block(suffix, block, flow)
            if store == False: actual_lines.append(theline)
            else: store.append(theline)
        block['implemented'] = True

In [9]:
def compile_block_flow(flow, suffix=''):
    global done
    prev_block_id = None
    for block in flow:
        if done == True: break
        compile_block(block, flow, prev_block_id, suffix)
        prev_block_id = block['id']

def loop_compiler(loop, loops):
    global blocks, done, actual_lines
    done = False
    actual_lines.append('def loop'+str(loops.index(loop))+'():\n')
    flow = [blocks[block] for block in loop['items']]
    for block in flow: block['implemented'] = False
    compile_block_flow(flow, '\t')

def flow_compiler(blocks, loops, the_start_block_id):
    global actual_lines
    starting_value = blocks[the_start_block_id]['value']
    actual_lines.append(starting_value)
    actual_lines.append("\n" if len(starting_value) > 0 else '')

    compile_block_flow(generate_flow(the_start_block_id))

    for loop in loops: loop_compiler(loop, loops)

# The function

In [10]:
import xml.etree.ElementTree as ET
# Load variables from .env file into the environment
load_dotenv()

dealing_decisions = []
blocks = {}
loops = []
loop_refered = False
decision_refered = False
in_loop=False
in_decision=False
done = False
actual_lines = []
decisions = {}

In [11]:
# def convert():

In [12]:
# 1. Read the Draw.io file
with open(os.getenv("SRC_FILE"), "r") as file:
    drawio_data = file.read()

In [13]:
# 2. Extract the XML data
xml_data = prepare_xml(drawio_data)
root = ET.fromstring(xml_data)

if len(os.getenv("DEST_XML_FILE")) >= 0:
    with open(os.getenv("DEST_XML_FILE"), "w") as file:
        file.writelines(xml_data)

In [14]:
# 3. Process the XML data
blocks, loops, the_start_block_id = compile_shapes(root)

/tmp/ipykernel_20956/378887861.py:53: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  look_for_block_in_previous_loops = len(np.where(np.array(blocks_iterated) == block_id)[0])


In [15]:
lines = flow_compiler(blocks, loops, the_start_block_id)

sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sasho
sash

KeyboardInterrupt: 

In [ ]:
with open(os.getenv("DEST_FILE"), "w") as file:
    file.writelines(lines)